In [8]:
import pandas as pd
import numpy as np
import string

In [9]:
products=pd.read_csv('amazon_baby_subset.csv')

In [10]:
products=products.fillna({'review':''})

In [32]:
def remove_punc(text):
    translator=str.maketrans({k:'' for k in string.punctuation})
    return text.translate(translator)
products['review_clean']=products['review'].apply(remove_punc)
important_words=pd.read_json('important_words.json')
important_words=important_words[0].values.tolist()

In [33]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [34]:
products.head()

,name,review,rating,sentiment,review_clean,0,baby,one,great,love,...,seems,picture,completely,wish,buying,babies,won,tub,almost,either
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [35]:
train_index=pd.read_json('module-4-assignment-train-idx.json')

In [36]:
valid_index=pd.read_json('module-4-assignment-validation-idx.json')

In [42]:
train_data=products.loc[train_index[0].tolist()]
valid_data=products.loc[valid_index[0].tolist()]

In [50]:
def get_numpy_data(dataframe,features,output):
    dataframe['constant']=1
    features=['constant']+features
    feature_matrix=dataframe[features].values
    output_array=dataframe[output].values
    return (feature_matrix,output_array)

In [52]:
feat_mat_train,sentiment_train=get_numpy_data(train_data,important_words,'sentiment')
feat_mat_valid,sentiment_valid=get_numpy_data(valid_data,important_words,'sentiment')

In [86]:
def predict_prob(feature_matrix,coefficients):
    score=np.dot(feature_matrix,coefficients)
    probability=1/(1+np.exp(-score))
    return probability

In [87]:
def cal_derivative(feature, errors, coefficient, l2_penalty, feature_is_constant):
    deriv_no=np.dot(feature,errors)
    if feature_is_constant==False:
        derivative=deriv_no-2*l2_penalty*coefficient
    else:
        derivative=deriv_no
    return derivative

def feature_derivative_with_L2feature (errors, feature, coefficient, l2_penalty, feature_is_constant): 
    
    # Compute the dot product of errors and feature
    ## YOUR CODE HERE
    derivative = np.dot(errors,feature)
    # add L2 penalty term for any feature that isn't the intercept.
    if not feature_is_constant: 
        ## YOUR CODE HERE
        derivative =  derivative - 2*l2_penalty*coefficient
        
    return derivative

In [88]:
def compute_log_likelihood_with_L2(feature_matrix, sentiment, coefficients, l2_penalty):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores))) - l2_penalty*np.sum(coefficients[1:]**2)
    
    return lp

In [91]:
def logistic_regression_with_L2(feature_matrix, sentiment, initial_coefficients, step_size, l2_penalty, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        ## YOUR CODE HERE
        predictions = predict_prob(feature_matrix,coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in range(len(coefficients)): # loop over each coefficient
            is_intercept = (j == 0)
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            ## YOUR CODE HERE
            derivative = cal_derivative(errors, feature_matrix[:,j], coefficients[j], l2_penalty, is_intercept)
            # add the step size times the derivative to the current coefficient
            ## YOUR CODE HERE
            coefficients[j] += step_size*derivative
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood_with_L2(feature_matrix, sentiment, coefficients, l2_penalty)
            print ('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients


In [102]:
penalty0=logistic_regression_with_l2(feat_mat_train,sentiment_train,initial_coefficients=[0.0]*194,step_size=5e-6,l2_penalty=0,max_iter=501)

iteration   0: log likelihood of observed labels = -29179.39138303
iteration   1: log likelihood of observed labels = -29003.71259047
iteration   2: log likelihood of observed labels = -28834.66187288
iteration   3: log likelihood of observed labels = -28671.70781507
iteration   4: log likelihood of observed labels = -28514.43078198
iteration   5: log likelihood of observed labels = -28362.48344665
iteration   6: log likelihood of observed labels = -28215.56713122
iteration   7: log likelihood of observed labels = -28073.41743783
iteration   8: log likelihood of observed labels = -27935.79536396
iteration   9: log likelihood of observed labels = -27802.48168669
iteration  10: log likelihood of observed labels = -27673.27331484
iteration  11: log likelihood of observed labels = -27547.98083656
iteration  12: log likelihood of observed labels = -27426.42679977
iteration  13: log likelihood of observed labels = -27308.44444728
iteration  14: log likelihood of observed labels = -27193.8767

In [103]:
penalty4=logistic_regression_with_l2(feat_mat_train,sentiment_train,initial_coefficients=[0.0]*194,step_size=5e-6,l2_penalty=4,max_iter=501)

iteration   0: log likelihood of observed labels = -29179.39508175
iteration   1: log likelihood of observed labels = -29003.73417180
iteration   2: log likelihood of observed labels = -28834.71441858
iteration   3: log likelihood of observed labels = -28671.80345068
iteration   4: log likelihood of observed labels = -28514.58077957
iteration   5: log likelihood of observed labels = -28362.69830317
iteration   6: log likelihood of observed labels = -28215.85663259
iteration   7: log likelihood of observed labels = -28073.79071393
iteration   8: log likelihood of observed labels = -27936.26093762
iteration   9: log likelihood of observed labels = -27803.04751805
iteration  10: log likelihood of observed labels = -27673.94684207
iteration  11: log likelihood of observed labels = -27548.76901327
iteration  12: log likelihood of observed labels = -27427.33612958
iteration  13: log likelihood of observed labels = -27309.48101569
iteration  14: log likelihood of observed labels = -27195.0462

In [104]:
penalty10=logistic_regression_with_l2(feat_mat_train,sentiment_train,initial_coefficients=[0.0]*194,step_size=5e-6,l2_penalty=10,max_iter=501)

iteration   0: log likelihood of observed labels = -29179.40062984
iteration   1: log likelihood of observed labels = -29003.76654163
iteration   2: log likelihood of observed labels = -28834.79322654
iteration   3: log likelihood of observed labels = -28671.94687528
iteration   4: log likelihood of observed labels = -28514.80571589
iteration   5: log likelihood of observed labels = -28363.02048079
iteration   6: log likelihood of observed labels = -28216.29071186
iteration   7: log likelihood of observed labels = -28074.35036891
iteration   8: log likelihood of observed labels = -27936.95892966
iteration   9: log likelihood of observed labels = -27803.89576265
iteration  10: log likelihood of observed labels = -27674.95647005
iteration  11: log likelihood of observed labels = -27549.95042714
iteration  12: log likelihood of observed labels = -27428.69905549
iteration  13: log likelihood of observed labels = -27311.03455140
iteration  14: log likelihood of observed labels = -27196.7989

In [105]:
penalty1e2=logistic_regression_with_l2(feat_mat_train,sentiment_train,initial_coefficients=[0.0]*194,step_size=5e-6,l2_penalty=1e2,max_iter=501)

iteration   0: log likelihood of observed labels = -29179.48385120
iteration   1: log likelihood of observed labels = -29004.25177457
iteration   2: log likelihood of observed labels = -28835.97382190
iteration   3: log likelihood of observed labels = -28674.09410083
iteration   4: log likelihood of observed labels = -28518.17112932
iteration   5: log likelihood of observed labels = -28367.83774654
iteration   6: log likelihood of observed labels = -28222.77708939
iteration   7: log likelihood of observed labels = -28082.70799392
iteration   8: log likelihood of observed labels = -27947.37595368
iteration   9: log likelihood of observed labels = -27816.54738615
iteration  10: log likelihood of observed labels = -27690.00588850
iteration  11: log likelihood of observed labels = -27567.54970126
iteration  12: log likelihood of observed labels = -27448.98991327
iteration  13: log likelihood of observed labels = -27334.14912742
iteration  14: log likelihood of observed labels = -27222.8604

In [106]:
penalty1e3=logistic_regression_with_l2(feat_mat_train,sentiment_train,initial_coefficients=[0.0]*194,step_size=5e-6,l2_penalty=1e3,max_iter=501)

iteration   0: log likelihood of observed labels = -29180.31606471
iteration   1: log likelihood of observed labels = -29009.07176112
iteration   2: log likelihood of observed labels = -28847.62378912
iteration   3: log likelihood of observed labels = -28695.14439397
iteration   4: log likelihood of observed labels = -28550.95060743
iteration   5: log likelihood of observed labels = -28414.45771129
iteration   6: log likelihood of observed labels = -28285.15124375
iteration   7: log likelihood of observed labels = -28162.56976044
iteration   8: log likelihood of observed labels = -28046.29387744
iteration   9: log likelihood of observed labels = -27935.93902900
iteration  10: log likelihood of observed labels = -27831.15045502
iteration  11: log likelihood of observed labels = -27731.59955260
iteration  12: log likelihood of observed labels = -27636.98108219
iteration  13: log likelihood of observed labels = -27547.01092670
iteration  14: log likelihood of observed labels = -27461.4242

In [107]:
penalty1e5=logistic_regression_with_l2(feat_mat_train,sentiment_train,initial_coefficients=[0.0]*194,step_size=5e-6,l2_penalty=1e5,max_iter=501)

iteration   0: log likelihood of observed labels = -29271.85955115
iteration   1: log likelihood of observed labels = -29271.71006589
iteration   2: log likelihood of observed labels = -29271.65738833
iteration   3: log likelihood of observed labels = -29271.61189923
iteration   4: log likelihood of observed labels = -29271.57079975
iteration   5: log likelihood of observed labels = -29271.53358505
iteration   6: log likelihood of observed labels = -29271.49988440
iteration   7: log likelihood of observed labels = -29271.46936584
iteration   8: log likelihood of observed labels = -29271.44172890
iteration   9: log likelihood of observed labels = -29271.41670149
iteration  10: log likelihood of observed labels = -29271.39403722
iteration  11: log likelihood of observed labels = -29271.37351294
iteration  12: log likelihood of observed labels = -29271.35492661
iteration  13: log likelihood of observed labels = -29271.33809523
iteration  14: log likelihood of observed labels = -29271.3228

In [127]:
words=pd.DataFrame({'word':['intercept']+important_words})

In [159]:
words['p0']=penalty0
words['p4']=penalty4
words['p10']=penalty10
words['p1e2']=penalty1e2
words['p1e3']=penalty1e3
words['p1e5']=penalty1e5
positive_words=words.sort_values('p0',ascending=False)['word'][0:5]
negative_words=words.sort_values('p0',ascending=True)['word'][0:5]
table_positive_words = words[words['word'].isin(positive_words)]
table_negative_words=words[words['word'].isin(negative_words)]
table_positive_words.sort_values('p0')


,word,p0,p4,p10,p1e2,p1e3,p1e5
3,great,0.801625,0.796897,0.789935,0.701425,0.376012,0.008950
34,perfect,0.835693,0.828555,0.818038,0.684143,0.250614,0.003989
8,easy,0.984559,0.977600,0.967362,0.838245,0.401904,0.008808
23,loves,1.052484,1.043903,1.031265,0.870794,0.345870,0.006150
4,love,1.058554,1.050856,1.039529,0.896644,0.418354,0.009042


In [158]:
table_negative_words

,word,p0,p4,p10,p1e2,p1e3,p1e5
97,money,-0.768793,-0.762734,-0.753818,-0.641406,-0.275883,-0.005487
106,disappointed,-0.955437,-0.946980,-0.934518,-0.775625,-0.266095,-0.004013
113,waste,-0.617809,-0.612475,-0.604620,-0.505189,-0.190631,-0.003345
114,return,-0.742085,-0.735502,-0.725807,-0.602646,-0.215199,-0.003730
169,returned,-0.572707,-0.567518,-0.559870,-0.462056,-0.150021,-0.002225


In [166]:
pre0=predict_prob(feat_mat_train,penalty0)
pre4=predict_prob(feat_mat_train,penalty4)
pre10=predict_prob(feat_mat_train,penalty10)
pre1e2=predict_prob(feat_mat_train,penalty1e2)
pre1e3=predict_prob(feat_mat_train,penalty1e3)
pre1e5=predict_prob(feat_mat_train,penalty1e5)


In [167]:
train_data['p0']=pre0>0.5
train_data['p4']=pre4>0.5
train_data['p10']=pre10>0.5
train_data['p1e2']=pre1e2>0.5
train_data['p1e3']=pre1e3>0.5
train_data['p1e5']=pre1e5>0.5

In [170]:
train_data['output']=train_data['sentiment']==1

In [171]:
train_data.head()

,name,review,rating,sentiment,review_clean,0,baby,one,great,love,...,almost,either,constant,p0,p4,p10,p1e2,p1e3,p1e5,output
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,0,1,0,...,0,0,1,True,True,True,True,True,True,True
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,1,False,False,False,False,False,True,True
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...,0,0,0,0,0,...,0,0,1,True,True,True,True,True,True,True
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...,0,0,0,1,0,...,0,0,1,True,True,True,True,True,True,True
5,Our Baby Girl Memory Book,"Beautiful book, I love it to record cherished ...",5,1,Beautiful book I love it to record cherished t...,0,0,0,1,1,...,0,0,1,True,True,True,True,True,True,True


In [173]:
np.sum(train_data['p0']==train_data['output'])

33260

In [174]:
np.sum(train_data['p4']==train_data['output'])

33258

In [175]:
np.sum(train_data['p10']==train_data['output'])

33253

In [176]:
np.sum(train_data['p1e2']==train_data['output'])

33210

In [177]:
np.sum(train_data['p1e3']==train_data['output'])

32866

In [178]:
np.sum(train_data['p1e5']==train_data['output'])

28821

In [179]:
pre0=predict_prob(feat_mat_valid,penalty0)
pre4=predict_prob(feat_mat_valid,penalty4)
pre10=predict_prob(feat_mat_valid,penalty10)
pre1e2=predict_prob(feat_mat_valid,penalty1e2)
pre1e3=predict_prob(feat_mat_valid,penalty1e3)
pre1e5=predict_prob(feat_mat_valid,penalty1e5)
valid_data['p0']=pre0>0.5
valid_data['p4']=pre4>0.5
valid_data['p10']=pre10>0.5
valid_data['p1e2']=pre1e2>0.5
valid_data['p1e3']=pre1e3>0.5
valid_data['p1e5']=pre1e5>0.5

In [180]:
valid_data['output']=valid_data['sentiment']==1

In [181]:
valid_data.head()

,name,review,rating,sentiment,review_clean,0,baby,one,great,love,...,almost,either,constant,p0,p4,p10,p1e2,p1e3,p1e5,output
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...,0,1,0,0,0,...,0,0,1,True,True,True,True,True,True,True
9,Cloth Diaper Pins Stainless Steel Traditional ...,It has been many years since we needed diaper ...,5,1,It has been many years since we needed diaper ...,0,0,1,0,0,...,0,0,1,False,False,False,False,False,True,True
23,Fisher Price Nesting Action Vehicles,For well over a year my son has enjoyed stacki...,5,1,For well over a year my son has enjoyed stacki...,0,0,1,0,0,...,0,0,1,True,True,True,True,True,True,True
26,Sassy Who Loves Baby? Photo Album Book with te...,I bought this for a new granddaughter. I will...,5,1,I bought this for a new granddaughter I will ...,0,0,0,2,0,...,0,0,1,True,True,True,True,True,True,True
27,Earlyears: Earl E. Bird with Teething Rings,We received an Earl E. Bird as a gift when we ...,5,1,We received an Earl E Bird as a gift when we h...,0,4,0,1,0,...,0,0,1,True,True,True,True,True,True,True


In [182]:
np.sum(valid_data['output']==valid_data['p0'])

8370

In [183]:
np.sum(valid_data['output']==valid_data['p4'])

8371

In [184]:
np.sum(valid_data['output']==valid_data['p10'])

8373

In [185]:
np.sum(valid_data['output']==valid_data['p1e2'])

8366

In [186]:
np.sum(valid_data['output']==valid_data['p1e3'])

8262

In [187]:
np.sum(valid_data['output']==valid_data['p1e5'])

7153